# Analisi della stagionalità

# importazione pacchetti

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle

# Apertura file pickle: Cambiare file path

In [ ]:
# 1. Inserisci qui il percorso del file pickle
file_path = 'cluster_dicts/cluster_dict_k-means_euristics_2.pkl'

###############################

# 2. Apri e carica il contenuto del file pickle in modalità binaria
with open(file_path, 'rb') as file:
    data = pickle.load(file)

###############################

# 3. Stampa il tipo di oggetto caricato (es. dict, list, altro)
print(f'Tipo oggetto caricato: {type(data)}')

###############################

# 4. Se l'oggetto caricato è un dizionario, stampa le prime 10 chiavi
#    Se è una lista, stampa i primi 5 elementi
if isinstance(data, dict):
    print(f'Chiavi: {list(data.keys())[:10]}')  # prime 10 chiavi
elif isinstance(data, list):
    print(f'Primi elementi: {data[:5]}')

###############################

# 5. Stampa il contenuto completo o il riassunto dell'oggetto caricato
print(data)

###############################


In [ ]:
print(data)

In [ ]:
# 1. Caricamento e copia del file CSV in DataFrame
delivery_data = pd.read_csv('delivery_history.csv').copy()

# 2. Conversione della colonna 'delivery_date' in datetime, con gestione errori
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')

# 3. Estrazione della sola data senza orario in nuova colonna 'delivery_day'
delivery_data['delivery_day'] = delivery_data['delivery_date'].dt.date

# 4. Estrazione del giorno della settimana come numero (0=lunedì, ..., 6=domenica)
delivery_data['weekday_num'] = delivery_data['delivery_date'].dt.dayofweek

# 5. Filtraggio DataFrame mantenendo solo i giorni feriali (lun-ven)
delivery_data = delivery_data[delivery_data['weekday_num'] < 5]

# 6. Rimozione righe con date non valide (NaN in 'delivery_day')
delivery_data = delivery_data.dropna(subset=['delivery_day'])

# 7. Definizione dizionario di mappatura giorno_num -> nome abbreviato italiano
weekday_map = {0:'Lun', 1:'Mar', 2:'Mer', 3:'Gio', 4:'Ven'}

# 8. Mappatura del numero giorno in nome giorno in colonna 'weekday_name'
delivery_data['weekday_name'] = delivery_data['weekday_num'].map(weekday_map)

# 9. Estrazione del mese da 'delivery_date' in colonna 'month'
delivery_data['month'] = delivery_data['delivery_date'].dt.month

# 10. Estrazione dell'anno da 'delivery_date' in colonna 'year'
delivery_data['year'] = delivery_data['delivery_date'].dt.year

# 11. Creazione colonna 'year_month' in formato periodo mensile (tipo Period)
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

# 12. Creazione maschera per valori validi in 'is_event' (0,1,True,False)
valid_bool_mask = delivery_data['is_event'].isin([0, 1, True, False])

# 13. Conversione in booleano della colonna 'is_event' solo per valori validi
delivery_data.loc[valid_bool_mask, 'is_event'] = delivery_data.loc[valid_bool_mask, 'is_event'].astype(bool)

# 14. Filtraggio righe mantenendo solo eventi con 'is_event' == True
delivery_data = delivery_data[delivery_data['is_event'] == True].copy()


cluster_data = data


# 15. Creazione dizionario vuoto per mappare location_id a cluster_id

location_to_cluster = {}
for cluster_id, locations in cluster_data.items():
    for loc in locations:
        location_to_cluster[loc] = cluster_id
# 17. Mappatura della colonna 'location_id' in 'cluster' usando il dizionario location_to_cluster
delivery_data['cluster'] = delivery_data['location_id'].map(location_to_cluster)

# 18. Rimozione delle righe senza mapping valido in 'cluster' (NaN)
delivery_data = delivery_data.dropna(subset=['cluster'])

# 19. Conversione della colonna 'cluster' a tipo intero
delivery_data['cluster'] = delivery_data['cluster'].astype(int)


In [ ]:
print('Num cluster unici:', delivery_data['cluster'].nunique())
print('Numero righe:', len(delivery_data))


# Calcolo numero di location_id attivi per cluster,mese e giorno

In [ ]:
# 1. Calcola il numero giornaliero unico di location_id attivi per cluster, mese e giorno
daily_active_points = delivery_data.groupby(
    ['cluster', 'year_month', 'delivery_day']
)['location_id'].nunique().reset_index(name='daily_active_points_count')

# 2. Calcola, per ogni cluster e mese, la media, minimo e massimo dei punti attivi giornalieri
stats_daily = daily_active_points.groupby(['cluster', 'year_month'])['daily_active_points_count'].agg(
    ['mean', 'min', 'max']
).reset_index()

# 3. Definisce una funzione per formattare le etichette nella heatmap: mostra la media e tra parentesi minimo e massimo
def format_label(row):
    mean_val = int(round(row['mean']))
    return f"{mean_val}\n({row['min']}; {row['max']})"

# 4. Applica la formattazione delle etichette su ciascuna riga del DataFrame delle statistiche
stats_daily['label'] = stats_daily.apply(format_label, axis=1)

# 5. Crea una tabella pivot per la media giornaliera (valori numerici) da usare come base per la heatmap
pivot_mean_daily = stats_daily.pivot(index='cluster', columns='year_month', values='mean').fillna(0)

# 6. Crea una tabella pivot delle etichette formattate da posizionare come annotazioni sulla heatmap
pivot_labels_daily = stats_daily.pivot(index='cluster', columns='year_month', values='label').fillna("")

# 7. Imposta la figura del grafico con dimensioni personalizzate
plt.figure(figsize=(20, 18))

# 8. Crea la heatmap con cmap 'Blues', aggiunge le annotazioni personalizzate e barra colore etichettata
sns.heatmap(pivot_mean_daily, annot=pivot_labels_daily, fmt='', cmap='Blues', cbar_kws={'label': 'Media punti attivi giornalieri'})

# 9. Imposta titolo, etichette assi e rotazione degli xticks e yticks per migliore leggibilità
plt.title('Media giornaliera di punti attivi per cluster e mese\n(con minimo e massimo tra parentesi)')
plt.xlabel('Mese')
plt.ylabel('Cluster')
plt.xticks(rotation=45)
plt.yticks(rotation=0)

# 10. Adatta il layout e mostra il grafico
plt.tight_layout()
plt.show()


## boxplot delle varie medie di n di location_id attivi per giorno e mese per cluster

In [ ]:
# Trasforma la colonna 'year_month' in stringa per etichette più leggibili
daily_active_points['year_month_str'] = daily_active_points['year_month'].astype(str)

# Crea una griglia di boxplot, uno per ciascun cluster, con più colonne (4 per riga)
g = sns.catplot(
    data=daily_active_points,                     # Dati da visualizzare
    x='year_month_str',                           # Asse X: mese (stringa)
    y='daily_active_points_count',                # Asse Y: numero di punti attivi giornalieri
    col='cluster',                                # Istanzia un boxplot per ogni cluster in colonne diverse
    kind='box',                                   # Tipo di grafico: boxplot
    col_wrap=4,                                   # Numero massimo di plot per riga
    height=4,                                     # Altezza di ciascun plot
    aspect=1.2,                                   # Rapporto lato larga/lato alto
    sharey=False                                  # Asse Y indipendente per ogni plot (per distribuzioni diverse)
)

# Imposta le etichette sugli assi
g.set_axis_labels("Mese", "Punti attivi giornalieri")
# Imposta i titoli per ogni plot con il nome del cluster
g.set_titles("Cluster {col_name}")
# Ruota le etichette dell'asse x di 45° per maggiore leggibilità
g.set_xticklabels(rotation=45)

# Aggiusta il layout, posiziona il titolo principale e mostra il grafico
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Distribuzione giornaliera punti attivi per cluster e mese (boxplot)')
plt.show()


# Test SEASONALITY 

In [ ]:
# Funzione per testare se ci sono differenze stagionali nei punti attivi per un singolo cluster
def test_seasonality_cluster(df_cluster):
    # Raggruppa i dati per mese e crea una lista di array contenenti i valori giornalieri per ogni mese
    groups = [group['daily_active_points_count'].values for name, group in df_cluster.groupby('year_month')]
    
    # Esegue il test ANOVA one-way per confrontare le medie tra i mesi
    anova_result = stats.f_oneway(*groups)
    
    # Esegue il test non parametrico di Kruskal-Wallis, utile se i dati non soddisfano le ipotesi di ANOVA
    kruskal_result = stats.kruskal(*groups)
    
    # Restituisce un dizionario con statistiche e p-value di entrambi i test
    return {
        'anova_stat': anova_result.statistic,
        'anova_pvalue': anova_result.pvalue,
        'kruskal_stat': kruskal_result.statistic,
        'kruskal_pvalue': kruskal_result.pvalue
    }

# Applica i test a tutti i cluster del dataset
results = []
for cluster, df_c in daily_active_points.groupby('cluster'):
    # Esegue la funzione di test per ogni singolo cluster
    test_res = test_seasonality_cluster(df_c)
    # Aggiunge il numero del cluster ai risultati
    test_res['cluster'] = cluster
    # Inserisce il risultato nella lista
    results.append(test_res)

# Crea un DataFrame con i risultati di tutti i cluster
results_df = pd.DataFrame(results)

# Stampa i risultati dei test di significatività per ogni cluster
print(results_df)


In [ ]:
# Filtra risultati con p-value significativo almeno in uno dei due test
significant_clusters = results_df[
    (results_df['anova_pvalue'] < 0.05) | (results_df['kruskal_pvalue'] < 0.05)
]

print(significant_clusters)


## Plot per quelli con p-value significativi

In [ ]:
# Filtra il DataFrame daily_active_points mantenendo solo i cluster significativi
significant_cluster_list = significant_clusters['cluster'].tolist()
filtered_data = daily_active_points[daily_active_points['cluster'].isin(significant_cluster_list)]

# Converte la colonna 'year_month' in stringa per una migliore visualizzazione sull'asse X
filtered_data['year_month_str'] = filtered_data['year_month'].astype(str)

# Crea un grafico a boxplot con Seaborn:
# - Un boxplot per ogni cluster significativo
# - Sull'asse X i mesi (in formato stringa)
# - Sull'asse Y il numero di punti attivi giornalieri
# - Dispone i plot in una griglia con 4 colonne per riga
g = sns.catplot(
    data=filtered_data,
    x='year_month_str',
    y='daily_active_points_count',
    col='cluster',
    kind='box',
    col_wrap=4,
    height=4,
    aspect=1.2,
    sharey=False  # asse Y autoscalato per ogni sottografo
)

# Imposta etichette per gli assi X e Y
g.set_axis_labels("Mese", "Punti attivi giornalieri")

# Imposta i titoli per ogni boxplot con il nome del cluster
g.set_titles("Cluster {col_name}")

# Ruota le etichette sull'asse X per migliorare la leggibilità
g.set_xticklabels(rotation=45)

# Aggiusta il layout per posizionare il titolo principale
plt.subplots_adjust(top=0.9)

# Aggiunge il titolo principale sopra la griglia di boxplot
g.fig.suptitle('Distribuzione giornaliera punti attivi per cluster e mese (boxplot) - solo significativi')

# Mostra il grafico
plt.show()
